In [1440]:
Auto_program_list=['HVC-062','IND-043','HVC-066']
Predict_list=['HVC-068','HVC-070','HVC-072' ]


In [1441]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDRegressor, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.inspection import permutation_importance

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor


#Program overview
This program predict the number of students that will be in AUT-052-5006 based on the number of students in AUT classes from the previous semester. AUT-052 was just a random class for auto majors in the 2nd semester. 
There is still a lot of automation to be done as this program moves to predicting all of the class in the 2nd semester instead of just one.

Conclusion: The PolynomialFeatures, LinearRegression, Pipeline using permutation importance can acuratly predict the students in AUT-052 based on the prevois semester students. It will be of great interest to see if this model can accurately predict the remain 2nd semester classes as well. 

In [1442]:
# read each sheet of the excel file
d={}
Semesters=('FA2017','SP2017','FA2018','SP2018','FA2019','SP2019','FA2020','SP2020','FA2021','SP2021', 'FA2022', 'SP2022')
for Semester in Semesters:
    d[Semester] = pd.read_excel('Data/Technology Student Counts.xlsx', sheet_name = Semester[2:]+ Semester[:2])
   

In [1443]:
#Lots of space padded data. Sometimes before, sometimes after.
#Get rid of EXCO, EXCL, and stdcnt blank data rows 

values = ['EXCO        ', 'EXCL        ', '       ']
for Semester in Semesters:
    d[Semester] =  d[Semester][ d[Semester]['Location    '].isin(values) == False]
    d[Semester] =  d[Semester][ d[Semester]['  StdCnt'].isin(values) == False]
    #print(Semester,d[Semester])

In [1461]:
##############################################test##################################################################
#*********************************************good********************************
#fall_df = pd.DataFrame({"AUT048":[0],"AUT050":[0], "AUT051":[0], "AUT080":[0]})
fall_df = pd.DataFrame()
keep_list=Auto_program_list # 
for Semester in Semesters:
    d[Semester+'d'] =  d[Semester].drop(['Location    ', 'CrseTypes   ','Depts ', '  Capacity'],
                                        axis=1).set_index('Section Name') #Remove unwanted columns
    d[Semester+'d'] = d[Semester+'d']['  StdCnt'].astype(str).astype(int) #Change StdCnt str to integer
    d[Semester+'d'] = pd.DataFrame(d[Semester+'d'])                       #Create new dataframes

    #print(d[Semester+'d'])#    if Semester[:2] == 'FA':                                              #If Fall semester rename and transform
    d['dt'+Semester] = d[Semester+'d'].T                         #  (add fdt prefix: fall, dropped, transformed)
    #print(d['dt'+Semester])                                                                     
        
#    if Semester[:2] == 'SP':                                              #If Spring semester rename and transform
#        d['sdt'+Semester] = d[Semester+'d'].T                             #  (add sdt prefix: spring, dropped, transformed)
        
        
#*********************************************good********************************   
    pred= d['dt'+Semester] # use this data to 
    test=pred.columns
    
    for i in range(len(test)):
        k =test[i][0:7]          # removes the section number from course number
        drop=0                   # resets whether to drop or not 
        #print(i,k)
        #print(Semester, test)
    if Semester[:2] == 'FA':
        keep_list=Auto_program_list
    else:
        keep_list=Predict_list
    for keep in (keep_list):
            if k in keep:         #If the class # (K) = a class on the list
                drop = drop+1     #don't drop
            #print(i,k,drop)
    if drop == 0:             # If the class number doesn't == to a class on the list drop it.
        #print('drop',test[i], test)  
        pred.drop(test[i], axis =1, inplace = True)
                #print(len(pred))
#break    # break here               
    #print(Semester, d['dt'+Semester]) #no spring data?############################################################
#*********************************************good********************************  
fall_df=pd.DataFrame()
for Semester in Semesters[:-3]: 
    #print(d['dtSP'+str(int(Semester[2:])+1)])
    #print(Semester, d['dt'+Semester])
    if Semester[:2] == 'FA':
        spring_s='SP'+str(int(Semester[2:])+1)
        #print(Semester, spring_s)
        #print(Semester, 'SP'+(int(Semester[2:])))
        size1=size2=size3=size4=size5=size6=0                       #sets size to 0
        HVC062=d['dt'+Semester].filter(regex='HVC-062',axis=1).sum()
        for i in range(len(HVC062)): #iterates through all of the columns that start with AUT048
            size1=size1+int(HVC062[i]) #filters out the columns that start with AUT-048 and sums them
              
        IND043=d['dt'+Semester].filter(regex='IND-043',axis=1).sum()
        for i in range(len(IND043)):
            size2=size2+int(IND043[i]) 
        HVC066=d['dt'+Semester].filter(regex='HVC-066',axis=1).sum()
        for i in range(len(HVC066)):
            size3=size3+int(HVC066[i]) 
#        AUT080=d['dt'+Semester].filter(regex='AUT-080',axis=1).sum()
#        for i in range(len(AUT080)):
#            size4=size4+int(AUT080[i]) 
        HVC072=d['dtSP'+str(int(Semester[2:])+1)].filter(regex='HVC-072',axis=1).sum()
        for i in range(len(HVC072)):
            size4=size4+int(HVC072[i])         
        HVC068=d['dtSP'+str(int(Semester[2:])+1)].filter(regex='HVC-068',axis=1).sum()
        #print(d['dtSP'+str(int(Semester[2:])+1)])
        #print(AUT052)
        for i in range(len(HVC068)):
            size5=size5+int(HVC068[i])
        HVC070=d['dtSP'+str(int(Semester[2:])+1)].filter(regex='HVC-070',axis=1).sum()
        for i in range(len(HVC070)):
            size6=size6+int(HVC070[i])    
        fall_df2=pd.DataFrame({"HVC-062":[size1], "IND-043":[size2], "HVC-066":[size3], "HVC-072":[size4], "HVC-068":[size5], "HVC-070":[size6]})    
        fall_df = fall_df.append(fall_df2)
        
fall_df

 


,HVC-062,IND-043,HVC-066,HVC-072,HVC-068,HVC-070
0,7,17,8,7,7,7
0,13,19,12,11,11,11
0,7,14,8,8,8,8
0,6,17,7,6,6,5
0,10,11,10,8,8,8


In [1386]:
#The program could read the data from the spread sheet, but that function is cuurentl
for Semester in Semesters[-2:-1]: 
    print(Semester)
    #print(Semester, d['dt'+Semester])
    if Semester[:2] == 'FA':
        #spring_s='SP'+str(int(Semester[2:])+1)
        #print(Semester, spring_s)
        #print(Semester, 'SP'+(int(Semester[2:])))
        size1=size2=size3=size4=0                       #sets size to 0
        HVC062=d['dt'+Semester].filter(regex='HVC-062',axis=1).sum()
        for i in range(len(HVC062)): #iterates through all of the columns that start with AUT048
            size1=size1+int(HVC062[i]) #filters out the columns that start with AUT-048 and sums them
              
        IND043=d['dt'+Semester].filter(regex='IND-043',axis=1).sum()
        for i in range(len(IND043)):
            size2=size2+int(IND043[i]) 
        HVC066=d['dt'+Semester].filter(regex='HVC-066',axis=1).sum()
        for i in range(len(HVC066)):
            size3=size3+int(HVC066[i]) 
#just set the class sizes manually for now                
size1=12
size2=14
size3=13
x_test=pd.DataFrame({"HVC-062":[size1], "IND-043":[size2], "HVC-066":[size3]})    
        
        
print(x_test)



FA2022
   HVC-062  IND-043  HVC-066
0       12       14       13


In [1391]:

Course_data_df=pd.DataFrame()
GBR = GradientBoostingRegressor()
parameters = {'learning_rate': [0.01,0.02,0.03,0.04],
                  'subsample'    : [0.9],
                  'n_estimators' : [100,500,1000, 1500],
                  'max_depth'    : [4,6,8,10]
                 }
old_courses=fall_df
Course_data_df=pd.DataFrame()
for course in Predict_list:
    X = old_courses.drop((Predict_list), axis =1)
    y = old_courses[course]
    X_train,  X_test, y_train, y_test = train_test_split(X,y, test_size = .125)
    grid_GBR = GridSearchCV(estimator=GBR, param_grid = parameters, cv = 2, n_jobs=-1)
    grid_GBR.fit(X_train, y_train)  

    print(" Results from Grid Search " )
    print("\n The best estimator across ALL searched params:\n",grid_GBR.best_estimator_)
    print("\n The best score across ALL searched params:\n",grid_GBR.best_score_)
    print("\n The best parameters across ALL searched params:\n",grid_GBR.best_params_)

    train_mse = mean_squared_error(y_train, grid_GBR.predict(X_train))
    test_mse = mean_squared_error(y_test, grid_GBR.predict(X_test))         
    cd_df=pd.DataFrame({'Course':[course], "train_mse":[train_mse], "test_mse":[test_mse], "predicted":[round(grid_GBR.predict(x_test)[0])]})
    Course_data_df=Course_data_df.append(cd_df)
#print('Course:',course,  'predicted =',round(pipe.predict(x_test)[0]))
#print('train_mse ',train_mse)
#print('test_mse ',test_mse)
#print(X_test) #print('Course:',course,'  Actual=', y_test[0], 'predicted =',pipe.predict(X_test)[0])
print(Course_data_df)
print('Based on these fall semester class numbers')
print(x_test)


 Results from Grid Search 

 The best estimator across ALL searched params:
 GradientBoostingRegressor(learning_rate=0.04, max_depth=10, subsample=0.9)

 The best score across ALL searched params:
 -1.6332023031506715

 The best parameters across ALL searched params:
 {'learning_rate': 0.04, 'max_depth': 10, 'n_estimators': 100, 'subsample': 0.9}
 Results from Grid Search 

 The best estimator across ALL searched params:
 GradientBoostingRegressor(learning_rate=0.04, max_depth=6, n_estimators=500,
                          subsample=0.9)

 The best score across ALL searched params:
 -0.015068528429084793

 The best parameters across ALL searched params:
 {'learning_rate': 0.04, 'max_depth': 6, 'n_estimators': 500, 'subsample': 0.9}
 Results from Grid Search 

 The best estimator across ALL searched params:
 GradientBoostingRegressor(learning_rate=0.01, max_depth=4, subsample=0.9)

 The best score across ALL searched params:
 -0.020000000000000018

 The best parameters across ALL search

In [1481]:

size=0

c={}
fall_df=pd.DataFrame()
for Semester in Semesters[:-3]: 
    #print(d['dtSP'+str(int(Semester[2:])+1)])
    #print(Semester, d['dt'+Semester])
    Courses=[]
    sizes=[]
    fall_df2=[]
    if Semester[:2] == 'FA':
        for class_num in Auto_program_list:
            #print(class_num[:3]+class_num[4:])
            c[class_num[:3]+class_num[4:]] =d['dt'+Semester].filter(regex=class_num,axis=1).sum()

            size=0
            for i in range(len(c[class_num[:3]+class_num[4:]])):
                    size=size+int(c[class_num[:3]+class_num[4:]][i])
                    #print(size)
            print(class_num[:3]+class_num[4:],'=',size)
            c[class_num] =class_num[:3]+class_num[4:],size
            Courses.append(class_num[:3]+class_num[4:])
            sizes.append(size)
        for class_num in Predict_list:
            #print(class_num[:3]+class_num[4:])
            c[class_num[:3]+class_num[4:]] =d['dtSP'+str(int(Semester[2:])+1)].filter(regex=class_num,axis=1).sum()
        #d['dtSP'+str(int(Semester[2:])+1)].filter(regex='HVC-072',axis=1).sum()   
            size=0
            for i in range(len(c[class_num[:3]+class_num[4:]])):
                    size=size+int(c[class_num[:3]+class_num[4:]][i])
                    #print(size)
            print(class_num[:3]+class_num[4:],'=',size)
            c[class_num] =class_num[:3]+class_num[4:],size
            Courses.append(class_num[:3]+class_num[4:])
            sizes.append(size)
  
        fall_df2 = pd.DataFrame(list(zip(Courses, sizes)), columns =['Section Name', 'StdCnt']).set_index('Section Name')
        fall_df2 = fall_df2.T
        fall_df=fall_df.append(fall_df2)
        print(fall_df2)
#fall_df2=fall_df2.T
print(fall_df)     #   fall_df = fall_df.append(fall_df2)

HVC062 = 7
IND043 = 17
HVC066 = 8
HVC068 = 7
HVC070 = 7
HVC072 = 7
Section Name  HVC062  IND043  HVC066  HVC068  HVC070  HVC072
StdCnt             7      17       8       7       7       7
HVC062 = 13
IND043 = 19
HVC066 = 12
HVC068 = 11
HVC070 = 11
HVC072 = 11
Section Name  HVC062  IND043  HVC066  HVC068  HVC070  HVC072
StdCnt            13      19      12      11      11      11
HVC062 = 7
IND043 = 14
HVC066 = 8
HVC068 = 8
HVC070 = 8
HVC072 = 8
Section Name  HVC062  IND043  HVC066  HVC068  HVC070  HVC072
StdCnt             7      14       8       8       8       8
HVC062 = 6
IND043 = 17
HVC066 = 7
HVC068 = 6
HVC070 = 5
HVC072 = 6
Section Name  HVC062  IND043  HVC066  HVC068  HVC070  HVC072
StdCnt             6      17       7       6       5       6
HVC062 = 10
IND043 = 11
HVC066 = 10
HVC068 = 8
HVC070 = 8
HVC072 = 8
Section Name  HVC062  IND043  HVC066  HVC068  HVC070  HVC072
StdCnt            10      11      10       8       8       8
Section Name  HVC062  IND043  HVC066  HVC068  HV